In [1]:
import pandas as pd
import geopy.distance
import gmaps
import requests
import json

In [2]:
df = pd.read_excel("data/Stores.xlsx")
df.head()

,Location Code,Address
0,JM1670,"345 Madison Ave New York, NY 10017"
1,JM1699,"200 Fifth Avenue New York, NY 10010"
2,JM1705,"74 Terminal C Newark, NJ 07114"
3,JM1847,"Madison Ave New York, NY 100225602"
4,JM1856,"American Dream East Rutherford, NJ 07073"


In [11]:
# Uses Google Maps Geocoding API for geographic coordinates
def get_coords(df):
    url = "https://maps.googleapis.com/maps/api/geocode/json?address="
    key = "&key=YOUR_API_KEY"
    d = []
    for index, row in df.iterrows():
        address = row['Address']
        location = row['Location Code']
        a = address.replace("#", "").replace(" ", "+")
        response = requests.get(url + a + key)
        payload = response.json()
        try:
            data = payload['results'][0]['geometry']['location']
        except IndexError:
            data = "NA"
        if data != "NA":
            lat = str(data['lat'])
            lng = str(data['lng'])
            d.append([location, address, lat, lng])

    return pd.DataFrame(d)

In [12]:
df2 = get_coords(df)
df2.to_excel("data/StoresCoords.xlsx", index=False)
df2.head()

,0,1,2,3
0,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065
1,JM1699,"200 Fifth Avenue New York, NY 10010",40.7419978,-73.989942
2,JM1705,"74 Terminal C Newark, NJ 07114",40.695605,-74.17714699999999
3,JM1847,"Madison Ave New York, NY 100225602",40.7788867,-73.9602326
4,JM1856,"American Dream East Rutherford, NJ 07073",40.8091063,-74.0698158


In [13]:
# Calculates straight-line distance
def get_distance(df):
    d = []
    for index, row in df.iterrows():
        location1 = row[0]
        address1 = row[1]
        latitude1 = row[2]
        longitude1 = row[3]
        coords1 = [latitude1, longitude1]
        for index, row in df.iterrows():
            location2 = row[0]
            address2 = row[1]
            latitude2 = row[2]
            longitude2 = row[3]
            coords2 = [latitude2, longitude2]
            distance = str(geopy.distance.geodesic(coords1, coords2).miles)
            if location1 != location2:
                d.append([location1, address1, latitude1, longitude1, location2, address2, latitude2, longitude2, distance])
    return pd.DataFrame(d)

In [14]:
df3 = get_distance(df2)
df3.to_excel("data/StoresDistances.xlsx", index=False)
df3.head()

,0,1,2,3,4,5,6,7,8
0,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1699,"200 Fifth Avenue New York, NY 10010",40.7419978,-73.989942,1.051496415105013
1,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1705,"74 Terminal C Newark, NJ 07114",40.695605,-74.17714699999999,11.213121897521512
2,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1847,"Madison Ave New York, NY 100225602",40.7788867,-73.9602326,1.940612781911012
3,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1856,"American Dream East Rutherford, NJ 07073",40.8091063,-74.0698158,6.132208434251751
4,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JN0409,"American Dream East Rutherford, NJ 07073",40.8091063,-74.0698158,6.132208434251751


In [15]:
# Filter for stores that are within 25 miles straight-line distance
df3[8] = df3[8].apply(pd.to_numeric)
df4 = df3[df3[8] <= 25]
df4.head()

,0,1,2,3,4,5,6,7,8
0,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1699,"200 Fifth Avenue New York, NY 10010",40.7419978,-73.989942,1.051496
1,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1705,"74 Terminal C Newark, NJ 07114",40.695605,-74.17714699999999,11.213122
2,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1847,"Madison Ave New York, NY 100225602",40.7788867,-73.9602326,1.940613
3,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JM1856,"American Dream East Rutherford, NJ 07073",40.8091063,-74.0698158,6.132208
4,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.9779065,JN0409,"American Dream East Rutherford, NJ 07073",40.8091063,-74.0698158,6.132208


In [16]:
# Uses Google Maps Distance Matrix API for estimated driving distance and duration
def get_driving_distance(df):
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins="
    url2 = "&destinations="
    key = "&key=YOUR_API_KEY"
    d = []
    for index, row in df.iterrows():
        location1 = row[0]
        address1 = row[1]
        latitude1 = str(row[2])
        longitude1 = str(row[3])
        location2 = row[4]
        address2 = row[5]
        latitude2 = str(row[6])
        longitude2 = str(row[7])
        response = requests.get(url + latitude1 + ',' + longitude1 + url2 + latitude2 + ',' + longitude2 + key)
        payload = response.json()
        try:
            data1 = payload['rows'][0]['elements'][0]['distance']
            distance = str(data1['text'])
        except KeyError:
            distance = "KeyError"
        except IndexError:
            distance = "IndexError"
        try:
            data2 = payload['rows'][0]['elements'][0]['duration']
            duration = str(data2['text'])
        except KeyError:
            duration = "KeyError"
        except IndexError:
            duration = "IndexError"
        d.append([location1, address1, latitude1, longitude1, location2, address2, latitude2, longitude2, 
                  distance, duration])
    df = pd.DataFrame(d)
    # Filter for instances where driving distance <= 25 miles
    df[10] = pd.to_numeric(df[8].str[:-3])
    df = df[df[10] <= 25]
    df = df.drop(columns=[10])
    # Format DataFrame
    df[2] = pd.to_numeric(df[2])
    df[3] = pd.to_numeric(df[3])
    df[6] = pd.to_numeric(df[6])
    df[7] = pd.to_numeric(df[7])
    df.rename(columns={0:'Location1',
                        1:'Address1',
                        2:'Latitude1', 
                        3:'Longitude1', 
                        4:'Location2', 
                        5:'Address2', 
                        6:'Latitude2', 
                        7:'Longitude2', 
                        8:'Distance', 
                        9:'Duration'},
               inplace=True)

    return df

In [17]:
df5 = get_driving_distance(df4)
df5.to_excel("data/StoresDrivingDistance.xlsx")
df5.head()

,Location1,Address1,Latitude1,Longitude1,Location2,Address2,Latitude2,Longitude2,Distance,Duration
0,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.977906,JM1699,"200 Fifth Avenue New York, NY 10010",40.741998,-73.989942,1.3 mi,10 mins
1,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.977906,JM1705,"74 Terminal C Newark, NJ 07114",40.695605,-74.177147,17.9 mi,35 mins
2,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.977906,JM1847,"Madison Ave New York, NY 100225602",40.778887,-73.960233,2.1 mi,15 mins
3,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.977906,JM1856,"American Dream East Rutherford, NJ 07073",40.809106,-74.069816,8.4 mi,22 mins
4,JM1670,"345 Madison Ave New York, NY 10017",40.754181,-73.977906,JN0409,"American Dream East Rutherford, NJ 07073",40.809106,-74.069816,8.4 mi,22 mins


In [18]:
store = input("Enter Location Code: ")

Enter Location Code: JM1670


In [19]:
# Uses gmaps API to plot stores on an interactive map
def plot_stores(store):
    gmaps.configure(api_key='YOUR_API_KEY')
    info_box_template = """
    <dl>
    <dt>Location Code:</dt><dd>{}</dd>
    <dt>Distance:</dt><dd>{}</dd>
    <dt>Duration:</dt><dd>{}</dd>
    </dl>
    """
    df = df5[df5['Location1'] == store]
    locations = []
    store_info = []

    for index, row in df.iterrows():
        latitude1 = row['Latitude1']
        longitude1 = row['Longitude1']
        location1 = row['Location1']
        latitude2 = row['Latitude2']
        longitude2 = row['Longitude2']
        location2 = row['Location2']
        distance = row['Distance']
        duration = row['Duration']
        locations.append((latitude2, longitude2))
        store_info.append(info_box_template.format(location2, distance, duration))

    center = (latitude1, longitude1)
    home = [(latitude1, longitude1), (latitude1, longitude1)]
    marker_locations = locations
    fig = gmaps.figure(center=center, zoom_level=11)

    markers = gmaps.marker_layer(marker_locations, info_box_content=store_info)
    symbols = gmaps.symbol_layer(home)

    fig.add_layer(markers)
    fig.add_layer(symbols)
    return fig

In [20]:
fig = plot_stores(store)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
df6 = df5[df5['Location1'] == store]
df6[['Location1', 'Address1', 'Location2', 'Address2', 'Distance', 'Duration']]

,Location1,Address1,Location2,Address2,Distance,Duration
0,JM1670,"345 Madison Ave New York, NY 10017",JM1699,"200 Fifth Avenue New York, NY 10010",1.3 mi,10 mins
1,JM1670,"345 Madison Ave New York, NY 10017",JM1705,"74 Terminal C Newark, NJ 07114",17.9 mi,35 mins
2,JM1670,"345 Madison Ave New York, NY 10017",JM1847,"Madison Ave New York, NY 100225602",2.1 mi,15 mins
3,JM1670,"345 Madison Ave New York, NY 10017",JM1856,"American Dream East Rutherford, NJ 07073",8.4 mi,22 mins
4,JM1670,"345 Madison Ave New York, NY 10017",JN0409,"American Dream East Rutherford, NJ 07073",8.4 mi,22 mins
5,JM1670,"345 Madison Ave New York, NY 10017",JM1861,"Willowbrook Mall Wayne, NJ 074706905",20.6 mi,37 mins
6,JM1670,"345 Madison Ave New York, NY 10017",JN0255,"626 Broadway New York, NY 10012",2.5 mi,17 mins
7,JM1670,"345 Madison Ave New York, NY 10017",JN0300,"34 West 34th Street New York, NY 100012946",0.8 mi,7 mins
8,JM1670,"345 Madison Ave New York, NY 10017",JN0336,"The Mall at Bay Plaza Bronx, NY 10475",14.8 mi,31 mins
9,JM1670,"345 Madison Ave New York, NY 10017",JN1036,"Queens Center Elmhurst, NY 11373",7.3 mi,20 mins
